# PhishEye

### Import Statements

In [ ]:
# pip install dnstwist
# pip install DNSPython
# pip install pillow

In [1]:
import pandas as pd
import dnstwist
import numpy as np 
import matplotlib.pyplot as plt 


### Dataset Exploration

In [188]:
fuzz = dnstwist.Fuzzer("www.google.com")
fuzz.generate()
len(fuzz.permutations())

2158

In [189]:
data = dnstwist.run(domain='google.com', registered=True, format='null')
reg = [d['domain'] for d in data]


In [190]:
data_non = dnstwist.run(domain='google.com', unregistered=True, format='null')
nonreg = [d['domain'] for d in data_non]


In [ ]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

intersection(reg, nonreg)

In [ ]:
print (f'The number of registered permutations is: {len(reg)}')
print (f'The number of non registered permutations is: {len(nonreg)}')


In [ ]:
print (reg[::15])
print (nonreg[::150])

In [ ]:
domains_df = pd.read_csv('./top-1m.csv', header=None, index_col=0)
domains_df.head()

In [ ]:
def get_dicts(domain_list):
    reg_dict, nonreg_dict = {}, {}
    for domain in domain_list:
        data_reg = dnstwist.run(domain=f'{domain}', registered=True, format='null')
        reg = [d['domain'] for d in data_reg]
        reg_dict[domain]  = len(reg)
        data_nonreg = dnstwist.run(domain=f'{domain}', unregistered=True, format='null')
        nonreg = [d['domain'] for d in data_nonreg]
        nonreg_dict[domain]  = len(nonreg)
    return reg_dict, nonreg_dict
# eda_reg, eda_nonreg = get_dicts(list(domains_df[1].values[:10]))
# eda_reg


In [ ]:
# x = list(eda_reg.keys())
# Yreg = list(eda_reg.values())
# Znonreg= list(eda_nonreg.values())
# X_axis = np.arange(len(x))
  
# plt.bar(x, Znonreg, color='steelblue')
# plt.bar(x, Yreg, bottom=Znonreg, color='darkorange')
  
# plt.xlabel("Domains")
# plt.ylabel("Number of Permutations")
# plt.title("Number of Registered and Non Registered Domain Permutations")
# plt.xticks(rotation=30)

# plt.legend(labels = ['Non Registered: Benign', 'Registered: Malicious'])
# plt.show()

In [ ]:
def create_twist_dict(domains):
    twist_dict = {}
    for domain in domains:
        #twist_dict[domain] = [[],[]]
        data_reg = dnstwist.run(domain=f'{domain}', registered=True, format='null')
        reg = [d['domain'] for d in data_reg]
        for homograph in reg:
            twist_dict[homograph] = [domain, True]

        data_nonreg = dnstwist.run(domain=f'{domain}', unregistered=True, format='null')
        nonreg = [d['domain'] for d in data_nonreg]
        for homograph in nonreg:
            twist_dict[homograph] = [domain, False]
    return twist_dict

twisted_dict = create_twist_dict(list(domains_df[1].values[:10]))

In [ ]:
twisted_df = pd.DataFrame.from_dict(twisted_dict, orient='index').reset_index()
twisted_df.columns = ['Homograph', 'Domain', 'Registered']
twisted_df.to_csv('twisted.csv')
twisted_df


### Text Distance Metrics

In [191]:
from strsimpy.levenshtein import Levenshtein
from strsimpy.jaro_winkler import JaroWinkler
from strsimpy.sorensen_dice import SorensenDice
from strsimpy.cosine import Cosine

test_string1 = reg[0] # google.com
test_string2 = nonreg[1] # g00qle.com

levenshtein = Levenshtein()
print(levenshtein.distance(test_string1, test_string2))

jarowinkler = JaroWinkler()
print(jarowinkler.distance(test_string1, test_string2))

sorensondice = SorensenDice()
print(sorensondice.distance(test_string1, test_string2))

cosine = Cosine(2)
a = cosine.get_profile(test_string1)
b = cosine.get_profile(test_string2)
print(cosine.similarity_profiles(a,b))


3
0.18000000000000005
0.5
0.5555555555555556


In [ ]:
for index, row in twisted_df.iterrows():
    twisted_df.loc[index,'Levenshtein'] = levenshtein.distance(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'Jaro-Winkler'] = jarowinkler.distance(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'Sorenson-Dice'] = sorensondice.distance(row['Domain'], row['Homograph'])
    str_to_vect_a= cosine.get_profile(row['Domain'])
    str_to_vect_b= cosine.get_profile(row['Homograph'])
    twisted_df.loc[index,'Cosine'] = cosine.similarity_profiles(str_to_vect_a, str_to_vect_b)

twisted_df    

In [ ]:
# twisted_df.to_csv('twisted_text_distance.csv')

### Image Similarity

In [4]:
from PIL import Image, ImageDraw, ImageFont

In [ ]:
# test sizing
lengths = [len(s) for s in twisted_df.Homograph]
longest_idx= lengths.index(max(lengths))
text = twisted_df.Homograph[longest_idx]
img = Image.new('RGB', (1024, 128))
# use bold font
font = ImageFont.truetype(f"./fonts/arial bold.ttf",70)
# draw image
d1 = ImageDraw.Draw(img)
# Center text in image
xpos = (img.size[0] / 2) - (font.getsize(text)[0]/2)
ypos = (img.size[1] / 2) - (font.getsize(text)[1]/2)
d1.text((xpos, ypos), text, fill =(255, 255, 255), font=font)
# show image
img.show()


In [3]:
import os.path

def create_image(string, font='arial.ttf', show=False):
    if not os.path.isfile(f'./images/{string}.jpeg'):
        img = Image.new('RGB', (1024, 128))
        text = string
        # use declared font
        font = ImageFont.truetype(f"./fonts/{font}",70)
        # draw image
        d1 = ImageDraw.Draw(img)
        # Center text in image
        xpos = (img.size[0] / 2) - (font.getsize(text)[0]/2)
        ypos = (img.size[1] / 2) - (font.getsize(text)[1]/2)
        d1.text((xpos, ypos), text, fill =(255, 255, 255), font=font)
        # show and save the image
        if show:
            img.show()
        img.save(f'images/{string}.jpeg')

# for test_string in [test_string1, test_string2]:
#     create_image(test_string, show=True)


### Simple

In [194]:
from skimage.metrics import structural_similarity as ssim, mean_squared_error as mse
import numpy as np
import cv2

def calculate_similarity(string_a, string_b):
    imageA = cv2.imread(f'./images/{string_a}.jpeg')
    imageB= cv2.imread(f'./images/{string_b}.jpeg')
    gsA = cv2.cvtColor(imageA, cv2.COLOR_BGR2GRAY)
    gsB = cv2.cvtColor(imageB, cv2.COLOR_BGR2GRAY)
    # Calculate the MSE, SSIM, COS
    m = mse(gsA, gsB)
    s = ssim(gsA, gsB)
    return  m, s
calculate_similarity(test_string1, test_string2)

0.028954753989188453

In [195]:
for index, row in twisted_df.iterrows():
    create_image(row['Domain'])
    create_image(row['Homograph'])
    m, s = calculate_similarity(row['Domain'], row['Homograph'])
    twisted_df.loc[index,'MSE'] = m
    twisted_df.loc[index,'SSIM'] = s

twisted_df


,Homograph,Domain,Registered,Levenshtein,Jaro-Winkler,Sorenson-Dice,Cosine,MSE,SSM,COS
0,google.com,google.com,True,0.0,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
1,google7.com,google.com,True,1.0,0.013774,0.294118,0.843274,3573.467743,0.848874,0.516013
2,googlea.com,google.com,True,1.0,0.013774,0.294118,0.843274,3562.883354,0.851078,0.551641
3,googled.com,google.com,True,1.0,0.013774,0.294118,0.843274,3576.705872,0.849858,0.531842
4,googlej.com,google.com,True,1.0,0.013774,0.294118,0.843274,3890.821388,0.849594,0.937457
...,...,...,...,...,...,...,...,...,...,...
47009,i.nstagram.com,instagram.com,False,1.0,0.022109,0.130435,0.880705,4724.995277,0.821886,0.839922
47010,in.stagram.com,instagram.com,False,1.0,0.020408,0.217391,0.880705,4730.435272,0.820904,0.864847
47011,inst.agram.com,instagram.com,False,1.0,0.071952,0.217391,0.880705,4727.817451,0.821145,0.848572
47012,insta.gram.com,instagram.com,False,1.0,0.048273,0.217391,0.880705,4748.178596,0.821017,0.808987


### Deep Learning

In [12]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms

# Load the pre-trained ResNet18 model
model = models.resnet18(pretrained=True)


# Define the preprocessing steps
preprocess = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define a function to extract features from an image using the pre-trained model
def extract_features(image):
    # Preprocess the image
    image = preprocess(image)
    # Add a batch dimension
    image = image.unsqueeze(0)
    # Pass the image through the model
    features = model(image)
    # Remove the batch dimension and return the feature vector
    return features.squeeze().detach().numpy()


def dl(i1, i2):
    
    # Load images from local directory
    image1 = Image.open(f'images/{i1}.jpeg').convert('L').resize((224, 224))
    image2 = Image.open(f'images/{i2}.jpeg').convert('L').resize((224, 224))

    # Apply transformations to images
    embeddings1 = extract_features(image1)  # add batch dimension
    embeddings2 = extract_features(image2)  # add batch dimension

    # Calculate cosine similarity between image features
    return  torch.nn.functional.cosine_similarity(torch.tensor(embeddings1), torch.tensor(embeddings2))
    
for index, row in twisted_df.head().iterrows():
    twisted_df.loc[index,'Swin'] = dl(row['Domain'], row['Homograph']).item()
    


/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [1]:
conda install numpy


failed

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


### Single Value Classifiers

In [196]:
twisted_df.to_csv('twisted_viz_sim_dl.csv')

In [2]:
import pandas as pd
twisted_df = pd.read_csv('twisted_viz_sim.csv', index_col=0)
twisted_df

,Homograph,Domain,Registered,Levenshtein,Jaro-Winkler,Sorenson-Dice,Cosine,MSE,SSM
0,google.com,google.com,True,0.0,0.000000,0.000000,1.000000,0.000000,1.000000
1,google7.com,google.com,True,1.0,0.013774,0.294118,0.843274,3573.467743,0.848874
2,googlea.com,google.com,True,1.0,0.013774,0.294118,0.843274,3562.883354,0.851078
3,googled.com,google.com,True,1.0,0.013774,0.294118,0.843274,3576.705872,0.849858
4,googlej.com,google.com,True,1.0,0.013774,0.294118,0.843274,3890.821388,0.849594
...,...,...,...,...,...,...,...,...,...
47009,i.nstagram.com,instagram.com,False,1.0,0.022109,0.130435,0.880705,4724.995277,0.821886
47010,in.stagram.com,instagram.com,False,1.0,0.020408,0.217391,0.880705,4730.435272,0.820904
47011,inst.agram.com,instagram.com,False,1.0,0.071952,0.217391,0.880705,4727.817451,0.821145
47012,insta.gram.com,instagram.com,False,1.0,0.048273,0.217391,0.880705,4748.178596,0.821017


In [201]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import export_text, DecisionTreeClassifier
import numpy as np

In [202]:
# Functions for running and validating supervised classification models
def model_eval(df, target, model):

    kf = StratifiedKFold(n_splits=5, shuffle=True)

    # Arrays to store metrics
    recall = []
    precision = []
    f1 = []

    for train_index, test_index in kf.split(df, target):
        X_train, X_test = df.iloc[train_index], df.iloc[test_index]
        y_train, y_test = target.iloc[train_index], target.iloc[test_index]
        model.fit(X_train, y_train)

        y_pred = model.predict(X_test)

        recall += [recall_score(y_pred, y_test, average="weighted", labels=[1])]
        precision += [precision_score(y_pred, y_test, average="weighted", labels=[1])]
        f1 += [f1_score(y_pred, y_test, average="weighted", labels=[1])]

    print("precision = {:.4f} ±{:.4f}".format(np.mean(precision), np.std(precision)))
    print("recall    = {:.4f} ±{:.4f}".format(np.mean(recall), np.std(recall)))
    print("f1        = {:.4f} ±{:.4f}".format(np.mean(f1), np.std(f1)))
    #return np.mean(f1)

In [203]:
# Add comment
dt_model = DecisionTreeClassifier(random_state=42, max_depth=1)
features = twisted_df.drop(['Domain', 'Homograph', 'Registered'], axis=1)
for col in features.columns:
    print (col)
    model_eval(twisted_df[[col]], twisted_df['Registered'] , dt_model)
    print()
    print(export_text(dt_model, feature_names=[col], decimals=10))


Levenshtein
precision = 0.8099 ±0.0343
recall    = 0.6706 ±0.0334
f1        = 0.7322 ±0.0095

|--- Levenshtein <= 2.5000000000
|   |--- class: True
|--- Levenshtein >  2.5000000000
|   |--- class: False

Jaro-Winkler
precision = 0.8245 ±0.0165
recall    = 0.6426 ±0.0239
f1        = 0.7221 ±0.0185

|--- Jaro-Winkler <= 0.1100206599
|   |--- class: True
|--- Jaro-Winkler >  0.1100206599
|   |--- class: False

Sorenson-Dice
precision = 0.8114 ±0.0247
recall    = 0.6450 ±0.0230
f1        = 0.7183 ±0.0165

|--- Sorenson-Dice <= 0.3798076957
|   |--- class: True
|--- Sorenson-Dice >  0.3798076957
|   |--- class: False

Cosine
precision = 0.8292 ±0.0228
recall    = 0.6627 ±0.0220
f1        = 0.7365 ±0.0205

|--- Cosine <= 0.7492276132
|   |--- class: False
|--- Cosine >  0.7492276132
|   |--- class: True

MSE
precision = 0.6428 ±0.0144
recall    = 0.5134 ±0.0118
f1        = 0.5708 ±0.0125

|--- MSE <= 3937.8059082031
|   |--- class: True
|--- MSE >  3937.8059082031
|   |--- class: False

SSM
